<h4>TextCNN</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from typing import Callable
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.python.profiler import profiler_v2 as profiler

2023-05-16 13:25:27.091262: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 13:25:27.368684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 13:25:28.508911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#loda all payloads from txt files in dataset folder (where each payload is one line)
def load_payloads(dataset_path: str) -> list:
    payloads = []
    for filename in os.listdir(dataset_path):
        with open(dataset_path + filename, 'r') as f:
            payloads.extend(f.readlines())
    return payloads

In [3]:
file_names=['malicious','normal']
for name in file_names:
    data=[]
    with jsonlines.open('dataset/'+name+'.txt') as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data.append({'request':{'method':'POST', 'uri':line['request']['uri'], 'body':line['request']['body'],'headers':line['request']['headers']},'metadata':line['metdata']})
            else:
                data.append({'request':{'method':'GET', 'uri':line['request']['uri'], 'headers':line['request']['headers']},'metadata':line['metdata']})
    with jsonlines.open('dataset/'+name+'_clean.txt', mode='w') as writer:
        writer.write_all(data)

<h4>Load and create datasets</h4>

In [4]:
from collections import Counter


file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))

payloads = load_payloads('dataset/payloads/')      
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

payload_iterator={}
payload_iterator[0] = 0
def get_next_payload():
    payload_iterator[0] = (payload_iterator[0] + 1) % len(payloads)
    return payloads[payload_iterator[0]]

#create synthetic malicious data by adding payloads to normal data (PAYLOAD SHOULD BE injected at a space character )
tmp = normal_part1.copy()
np.random.shuffle(payloads)
synthetic_malicious = []
for i in range(len(tmp)):
    payload = get_next_payload()
    tmp[i] = tmp[i].split(' ') 
    tmp[i].insert(np.random.randint(0,len(tmp[i])), payload)
    tmp[i] = ' '.join(tmp[i])
    synthetic_malicious.append(tmp[i])


# add shuffled malicious data to train set
#malicious_shuffled = malicious.copy()  
#np.random.shuffle(malicious_shuffled)
#train_examples.extend(malicious_shuffled[:50000])
#train_labels.extend([1] * len(malicious_shuffled[:50000]))

train_examples.extend(synthetic_malicious)
train_labels.extend([1] * len(synthetic_malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))

#shuflee datasets with given radnom seed
dataset_train = dataset_train.shuffle(400000, seed=42, reshuffle_each_iteration=False)
dataset_test = dataset_test.shuffle(400000, seed=42, reshuffle_each_iteration=False)

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())




2023-05-16 13:25:45.115058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 13:25:45.278638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 13:25:45.278688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 13:25:45.282013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 13:25:45.282083: I tensorflow/compile

b'a  / a / a-a  { "a-a": "a-a", "a": "a / a,a / a a,a / a;a = n.n,a / a,a / a,* / *;a = n.n", "a-a": "a", "a-a": "a / n.n  ( a; a a a a n_n_n )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0


2023-05-16 13:25:48.769888: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [211600]
	 [[{{node Placeholder/_1}}]]


b'a   / a / a - a   { " a - a " :   " a - a " ,   " a " :   " a / a , a / a   a , a / a ; a = n . n , a / a , a / a , * / * ; a = n . n " ,   " a - a " :   " a " ,   " a - a " :   " a / n . n   ( a ;   a   a   a   a   n _ n _ n )   a / n . n   ( a ,   a   a )   a / n . n . n . n   a / n . n " ,   " a " :   " a = x ;   a = x " } ' 0


<h4>Vocabularies</h4>

In [34]:

vectorization = TextVectorization(
    max_tokens=1000,
    output_mode="int", 
    output_sequence_length=256,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization_sub = TextVectorization(
    max_tokens=5000, 
    output_mode="int", 
    output_sequence_length=128,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization.adapt(train.map(lambda x, y: x))
vectorization_sub.adapt(train_sub.map(lambda x, y: x))

2023-04-23 00:43:23.850876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-23 00:53:16.487408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [5]:
#pickle.dump({'config': vectorization.get_config(),'weights': vectorization.get_weights()}, open("vectorization.pkl", "wb"))

#pickle.dump({'config': vectorization_sub.get_config(), 'weights': vectorization_sub.get_weights()}, open("vectorization_sub.pkl", "wb"))


from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-05-16 13:25:49.104957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-16 13:25:49.275249: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>CNN</h4>

In [ ]:
embedding_layer_sub = tf.keras.layers.Embedding(1000, 4)
embedding_layer = tf.keras.layers.Embedding(5000, 4)
flatten_layer = tf.keras.layers.Flatten()



def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

train_auto = train_auto.batch(32)
train_auto_sub = train_auto_sub.batch(32)


input_layer = tf.keras.layers.Input(shape=(256,))
mod = embedding_layer(input_layer)
mod =  layers.Conv1D(128, 16, activation='relu', padding='same')(mod)
mod = layers.MaxPooling1D(16, padding='same')(mod)
mod =  layers.Conv1D(128, 8, activation='relu', padding='same')(mod)
mod = layers.MaxPooling1D(8, padding='same')(mod)
mod = flatten_layer(mod)
mod =  tf.keras.layers.Dense(128, activation='relu')(mod)
mod =  tf.keras.layers.Dense(1, activation='sigmoid')(mod)

cnn = tf.keras.models.Model(input_layer, mod)

input_layer_sub = tf.keras.layers.Input(shape=(256,))
mod_sub = embedding_layer_sub(input_layer_sub)
mod_sub =  layers.Conv1D(128, 16, activation='relu', padding='same')(mod_sub)
mod_sub = layers.MaxPooling1D(16, padding='same')(mod_sub)
mod_sub =  layers.Conv1D(128, 8, activation='relu', padding='same')(mod_sub)
mod_sub = layers.MaxPooling1D(8, padding='same')(mod_sub)
mod_sub = flatten_layer(mod_sub)
mod_sub =  tf.keras.layers.Dense(128, activation='relu')(mod_sub)
mod_sub =  tf.keras.layers.Dense(1, activation='sigmoid')(mod_sub)

cnn_sub = tf.keras.models.Model(input_layer, mod_sub)

# Compile and train the model
cnn.compile(optimizer='adam',  loss=losses.BinaryCrossentropy())
cnn_sub.compile(optimizer='adam',  loss=losses.BinaryCrossentropy())


profiler.start(logdir='cnn_train')
cnn.fit(train_auto, epochs=15)
profiler.stop()
profiler.start(logdir='cnn_sub_train')
cnn_sub.fit(train_auto, epochs=15)
profiler.stop()

Epoch 1/15


2023-05-16 13:25:52.582477: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-05-16 13:25:52.582562: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2023-05-16 13:25:52.583541: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1679] Profiler found 1 GPUs
2023-05-16 13:25:52.610907: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:194] cuptiSubscribe: error 15: CUPTI_ERROR_NOT_INITIALIZED
2023-05-16 13:25:52.610951: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:459] cuptiGetResultString: ignored due to a previous error.
2023-05-16 13:25:52.610970: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1730] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2023-05-16 13:25:52.625415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (

6613/6613 [==============================] - 76s 10ms/step - loss: 0.0336
Epoch 2/15
6613/6613 [==============================] - 67s 10ms/step - loss: 0.0079
Epoch 3/15
6613/6613 [==============================] - 68s 10ms/step - loss: 0.0042
Epoch 4/15
6613/6613 [==============================] - 68s 10ms/step - loss: 0.0029
Epoch 5/15
2830/6613 [===========>..................] - ETA: 11:23 - loss: 0.0023

In [ ]:
#cnn.fit(train_auto, epochs=10)
cnn.save('cnn/my_model')
cnn = tf.keras.models.load_model('cnn/my_model')

cnn_sub.save('cnn_sub/my_model')
cnn_sub = tf.keras.models.load_model('cnn_sub/my_model')


In [40]:

test_auto_sub = test_sub.map(sub_processing)
test_auto_sub = test_auto_sub.batch(32)
test_auto = test.map(processing)
test_auto = test_auto.batch(32)
reconsturctions_sub =  cnn_sub.predict(test_auto_sub)
reconsturctions =  cnn.predict(test_auto)


2023-05-13 09:39:16.057311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [117899]
	 [[{{node Placeholder/_0}}]]


3685/3685 [==============================] - 26s 7ms/step


In [41]:
pred = list(map(lambda x: 0 if x[0]<0.5 else 1, list(reconsturctions)))
pred_sub = list(map(lambda x: 0 if x[0]<0.5 else 1, list(reconsturctions_sub)))

In [42]:
actual = test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))

2023-05-13 09:39:43.284034: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [117899]
	 [[{{node Placeholder/_0}}]]


In [43]:
print("Metrics")
print("f1: ", f1_score(actual,pred))
print("acc: ", accuracy_score(actual,pred))
print("prec: ", precision_score(actual,pred))

0.886868256946997

In [24]:
print("Metrics sub")
print("f1: ", f1_score(actual,pred_sub))
print("acc: ", accuracy_score(actual,pred_sub))
print("prec: ", precision_score(actual,pred_sub))

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
